In [ ]:
# default_exp train_loop

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# hide
from fastcore.test import *

# Training Loop

> Define the general Training Loop

In [ ]:
# export

# general
import torch
from tqdm import tqdm, tqdm_notebook

# import the relevant models
from deeptool.model.dcgan import DCGAN
from deeptool.model.diagnosis import TripleMRNet
from deeptool.model.vqvae import VQVAE2
from deeptool.model.introvae import IntroVAE
from deeptool.model.bigan import BiGAN
from deeptool.model.rnnvae import Creator_RNN_AE
from deeptool.model.mocoae import MoCoAE

# import the dataset
from deeptool.dataloader import (
    load_test_batch,
    load_kneexray_datasets,
    load_mrnet_datasets,
)

# import the parameters
from deeptool.parameters import get_all_args, compat_args

In [ ]:
# export


def get_model(device, args):
    """
    return the required model depending on the arguments:
    """
    print(f"Model-Type: {args.model_type}")
    switcher = {
        "dcgan": DCGAN,
        "diagnosis": TripleMRNet,
        "vqvae": VQVAE2,
        "introvae": IntroVAE,
        "rnnvae": Creator_RNN_AE,
        "bigan": BiGAN,
        "mocoae": MoCoAE,
    }
    # Get the model_creator
    model_creator = switcher.get(args.model_type, lambda: "Invalid Model Type")
    # create model
    return model_creator(device, args)

In [ ]:
# export


def get_dataset(args):
    """
    return the required datasets and dataloaders depending on the dataset
    """
    print(f"Dataset-Type: {args.dataset_type}")
    switcher = {
        "MRNet": load_mrnet_datasets,
        "KneeXray": load_kneexray_datasets,
    }
    # Get the model_creator
    dataset_loader = switcher.get(args.dataset_type, lambda: "Invalid Model Type")
    # create model
    return dataset_loader(args)

In [ ]:
# test model creation
args = get_all_args()
args.track = False
args.model_type = "bigan"
device = torch.device(
    "cuda:0" if (torch.cuda.is_available() and args.n_gpu > 0) else "cpu"
)
model = get_model(device, args)

Model-Type: bigan


In [ ]:
# export


def test_one_batch(args):
    """
    Useful functionality to test a new model using a demo databatch and check compatibility
    """
    args.track = False
    # define calculating device
    device = torch.device(
        "cuda:0" if (torch.cuda.is_available() and args.n_gpu > 0) else "cpu"
    )
    # avoid dataloaders for testing on server:
    batch = load_test_batch(args)
    # Initialise the Model
    model = get_model(device, args)
    # switch to train mode
    model.train()
    # evaluate batch
    model(batch)

In [ ]:
# export


def batch_info(test_x, args):
    if args.model_type == "diagnosis":
        print(test_x[args.perspectives[0]].shape)
        print(torch.min(test_x[args.perspectives[0]]))
        print(torch.max(test_x[args.perspectives[0]]))
    else:
        print(f"Input Dimension: {test_x.shape}")
        print(f"Minimum Value in Batch: {torch.min(test_x)}")
        print(f"Maximum Value in Batch: {torch.max(test_x)}")

In [ ]:
# export


def main_loop(args, tq_nb=True):
    """Perform the Training using the predefined arguments"""

    # select tqdm_type
    tq = tqdm
    if tq_nb:
        tq = tqdm_notebook

    # define calculating device
    device = torch.device(
        "cuda:0" if (torch.cuda.is_available() and args.n_gpu > 0) else "cpu"
    )

    # build the dataloaders
    train_data, _, train_loader, valid_loader = get_dataset(args)

    # Initialise the Model
    model = get_model(device, args)

    # test dataset
    test_data = next(iter(train_loader))
    test_x = model.prep(test_data)
    
    batch_info(test_x, args)

    # Load pretrained params
    if args.load_model:
        model.load_state_dict(torch.load(args.model_path))

    # start training
    print("\nStart training")
    batch_count = 0
    model.train()

    for epoch in tq(range(args.n_epochs)):

        # remodify after pretraining training
        if epoch == args.n_pretrain and args.model_type in ("vqvae", "introvae"):
            print("parameters set")
            model.set_parameters(args)

        # For each batch in the dataloader
        for data in tq(train_loader):
            # perform training
            model(data)

            if batch_count % args.watch_batch == 0:
                # eval status
                model.eval()

                if args.model_type in ("triplenet", "diagnosis"):
                    model.watch_progress(valid_loader, epoch)
                else:
                    model.watch_progress(test_data, epoch)

                # reset to train status again
                model.train()

            # watch progress
            batch_count += 1

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 05_abstract_model.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_bigan.ipynb.
Converted 24_mocoae.ipynb.
Converted 33_rnn_vae.ipynb.
Converted 99_index.ipynb.
